In [12]:
### RAG pipeline - Data Ingestion to Vector DB pipeline


In [13]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [14]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 6 PDF files to process

Processing: Bill's_Windsurf_Shop_Invoice.pdf
  ✓ Loaded 1 pages

Processing: Amy's_Bird_Sanctuary_Invoice.pdf
  ✓ Loaded 1 pages

Processing: Interim-Presentation.pdf
  ✓ Loaded 71 pages

Processing: Cool_Cars_Invoice.pdf
  ✓ Loaded 1 pages

Processing: Dukes_Basketball_Camp_Invoice.pdf
  ✓ Loaded 1 pages

Processing: Diego_Rodriguez_Invoice.pdf
  ✓ Loaded 1 pages

Total documents loaded: 76


In [15]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [16]:
chunks = split_documents(all_pdf_documents)
chunks

Split 76 documents into 93 chunks

Example chunk:
Content: Invoice for Bill's Windsurf Shop
Email: Surf@Intuit.com
Invoice Details:
Description Qty Unit Price Amount
Design Service 1 $500.00 $500.00
Consulting 2 $200.00 $400.00
Installation 1 $300.00 $300.00
...
Metadata: {'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'source': "../data/pdf_files/Bill's_Windsurf_Shop_Invoice.pdf", 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': "Bill's_Windsurf_Shop_Invoice.pdf", 'file_type': 'pdf'}


[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'source': "../data/pdf_files/Bill's_Windsurf_Shop_Invoice.pdf", 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': "Bill's_Windsurf_Shop_Invoice.pdf", 'file_type': 'pdf'}, page_content="Invoice for Bill's Windsurf Shop\nEmail: Surf@Intuit.com\nInvoice Details:\nDescription Qty Unit Price Amount\nDesign Service 1 $500.00 $500.00\nConsulting 2 $200.00 $400.00\nInstallation 1 $300.00 $300.00\nTotal $1200.00\nInvoice Date: October 20, 2025\nDue Date: November 19, 2025"),
 Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'source': "../data/pdf_files/Amy's_Bird_Sanctuary_Invoice.pdf", 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': "Amy's_Bird_Sanctuary_Invoice.pdf", 'file_type': 'pdf'}, page_content="Invoice for Amy's Bird Sanctuary\nEmail: Birds@Intuit.com\nInvoice Details:\nDescription Qty Unit Price Amount\nDesign Service 1 $500.00 $500.00\nConsulti

In [17]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()  # Loads the embedding model

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(
                f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}"
            )
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        Args:
            texts: List of text strings to embed
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings

## Initialize the embedding manager
embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [19]:
### VectorStore

In [20]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorStore = VectorStore()
vectorStore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [25]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]

### Generate the embeddings

embeddings = embedding_manager.generate_embeddings(texts)

## Store in the vector database
vectorStore.add_documents(chunks,embeddings)

Generating embeddings for 93 texts...


Batches: 100%|██████████| 3/3 [00:00<00:00,  6.16it/s]


Generated embeddings with shape: (93, 384)
Adding 93 documents to vector store...
Successfully added 93 documents to vector store
Total documents in collection: 93
